## import the data from the database 

In [1]:
#import the password from a seperate file that is ignored on github
# from config import password
# import dependencies 

import os 
from config import password 
from psycopg2 import sql, connect 
import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_covtype
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


In [2]:
try:
    # declare a new PostgreSQL connection object
    conn = connect(
        dbname = "",
        user = "postgres",
        host = "tennesseetornadoes.cdi1utmdgtwo.us-east-1.rds.amazonaws.com",
        port = "5432",
        password = password
    )
    # print the connection if successful
    print ("psycopg2 connection:", conn)
except Exception as err:
    print ("psycopg2 connect() ERROR:", err)
    conn = None

psycopg2 connection: <connection object at 0x7fa7fc6d7050; dsn: 'user=postgres password=xxx dbname='' host=tennesseetornadoes.cdi1utmdgtwo.us-east-1.rds.amazonaws.com port=5432', closed: 0>


In [3]:
cr = conn.cursor()
cr.execute('SELECT * FROM total_tn_tornadoes;')
tmp = cr.fetchall()

In [4]:
#extract the  column names
col_names = []
for db in cr.description:
    col_names.append(db[0])

In [5]:
# Create the dataframe, passing in the list of col_names extracted from the description
df = pd.DataFrame(tmp, columns=col_names)
df.dropna()

,yr,mo,date,time,mag,slat,slon,elat,elon,len,wid,fatalities,injuries,property_loss,starting_county,ending_county
0,2011,4,2011-04-09,14:35:00,0,36.2971,-82.3174,36.2934,-82.3021,0.89,50,0,0,4.0,Carter County,Carter County
1,2011,4,2011-04-09,16:00:00,1,36.3,-82.4341,36.3,-82.4341,3,150,0,0,5.0,Washington County,Washington County
2,2011,4,2011-04-09,16:08:00,1,36.3037,-82.3923,36.3099,-82.3846,0.61,100,0,0,4.0,Washington County,Washington County
3,2011,4,2011-04-09,16:15:00,0,36.0424,-82.5568,36.0477,-82.5393,1.04,50,0,0,4.0,Unicoi County,Unicoi County
4,2011,4,2011-04-27,19:07:00,0,36.2255,-83.057,36.2263,-83.0486,0.5,70,0,0,3.0,Greene County,Greene County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,2009,6,2009-06-12,16:00:00,1,35.1903,-89.8703,35.1907,-89.8685,0.1,30,0,0,4.0,Shelby County,Shelby County
1104,2009,7,2009-07-30,15:49:00,1,35.1864,-89.8086,35.1958,-89.7758,1.96,250,0,0,5.0,Shelby County,Shelby County
1107,2010,5,2010-05-01,21:47:00,1,35.3228,-89.9383,35.3259,-89.9282,0.61,100,0,0,4.0,Shelby County,Shelby County
1110,1991,11,1991-11-19,17:05:00,1,35.13,-90.02,35.28,-89.93,10,50,0,0,6.0,Shelby County,Shelby County


In [14]:
df = df.dropna()

### Run the first model

target = loss

yields approx 54% accuracy 

In [15]:
df_3 = df.drop(['time',"wid","starting_county",
              'slat','slon','elat','elon','len','ending_county'], axis=1)


In [90]:
# prepare the dataframe 

df_3 = df.drop(['time',"wid","starting_county",
              'slat','slon','elat','elon','len','ending_county'], axis=1)
# df_3.columns
# df_3['property_loss']=df_3['property_loss'].astype('int')

target = 'property_loss'
X = pd.get_dummies(df_3.drop([target],axis = 1))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

from sklearn.datasets import fetch_covtype
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_approximation import PolynomialCountSketch
from sklearn.linear_model import LogisticRegression

pipe = make_pipeline(
    MinMaxScaler(),
    PolynomialCountSketch(degree=2, n_components=300),
    LogisticRegression(max_iter=1000),
)


print(f"Accuracy Score: {round(pipe.fit(X_train, y_train).score(X_test, y_test)*100,2)}%")





Accuracy Score: 54.07%


### Run the second model 

target = Month 

yields approx 80% accuracy 

In [21]:
#sklearn.linear_model.SGDClassifier

df_1 = df.drop(['time','starting_county','fatalities','wid',
              'slat','slon','elat','elon','len','ending_county','injuries'], axis=1)

# Create our features

target = 'mo'
X = pd.get_dummies(df_1.drop(columns=target))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

pipe = make_pipeline(
    MinMaxScaler(),
    PolynomialCountSketch(degree=2, n_components=650),
    LogisticRegression(max_iter=1000),
)

print(f"Accuracy Score: {round(pipe.fit(X_train, y_train).score(X_test, y_test),3)*100}%")



Accuracy Score: 80.2%


### Run the third model 

target = Magnitude 


yields approx 55% accuracy 

Prepare the dataset 

In [133]:

#Keep Date, Starting county, property loss and mag

df_2 = df.drop([ 'time','date',
              'slat','slon','elat','elon','len','ending_county','starting_county','injuries'], axis=1)






df_2



,yr,mo,mag,wid,fatalities,property_loss
0,2011,4,0,50,0,4.0
1,2011,4,1,150,0,5.0
2,2011,4,1,100,0,4.0
3,2011,4,0,50,0,4.0
4,2011,4,0,70,0,3.0
...,...,...,...,...,...,...
1103,2009,6,1,30,0,4.0
1104,2009,7,1,250,0,5.0
1107,2010,5,1,100,0,4.0
1110,1991,11,1,50,0,6.0


In [140]:
# Create our features

target = 'mag'
X = pd.get_dummies(df_2.drop(columns=target))


# Create our target
y = df[target]


X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

pipe = make_pipeline(
    StandardScaler(),
    MinMaxScaler(),
    
    
    PolynomialCountSketch(degree=6, n_components=850),
    LogisticRegression(solver='sag',max_iter=1000),
)

print(f"Accuracy Score: {round(pipe.fit(X_train, y_train).score(X_test, y_test),3)*100}%")

Accuracy Score: 55.2%
